In [4]:
import pandas as pd

In [5]:
dfPath = "/content/melb_data.csv"
df = pd.read_csv(dfPath)
df.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


In [6]:
df.columns #finding all the columns present in dataframe

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [7]:
df = df.dropna(axis=0) # dropping all the null values in rows

In [8]:
df.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,6196.000000,6.196000e+03,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000
mean,2.931407,1.068828e+06,9.751097,3101.947708,2.902034,1.576340,1.573596,471.006940,141.568645,1964.081988,-37.807904,144.990201,7435.489509
std,0.971079,6.751564e+05,5.612065,86.421604,0.970055,0.711362,0.929947,897.449881,90.834824,38.105673,0.075850,0.099165,4337.698917
min,1.000000,1.310000e+05,0.000000,3000.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1196.000000,-38.164920,144.542370,389.000000
25%,2.000000,6.200000e+05,5.900000,3044.000000,2.000000,1.000000,1.000000,152.000000,91.000000,1940.000000,-37.855438,144.926198,4383.750000
50%,3.000000,8.800000e+05,9.000000,3081.000000,3.000000,1.000000,1.000000,373.000000,124.000000,1970.000000,-37.802250,144.995800,6567.000000
75%,4.000000,1.325000e+06,12.400000,3147.000000,3.000000,2.000000,2.000000,628.000000,170.000000,2000.000000,-37.758200,145.052700,10175.000000
max,8.000000,9.000000e+06,47.400000,3977.000000,9.000000,8.000000,10.000000,37000.000000,3112.000000,2018.000000,-37.457090,145.526350,21650.000000


In [9]:
y = df.Price #storing the predicting target

In [10]:
dFeatures = ['Rooms','Bathroom', 'Landsize', 'Lattitude', 'Longtitude'] #selecting features

In [11]:
x = df[dFeatures] #storing features data

In [12]:
x.describe()

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
count,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000
mean,2.931407,1.576340,471.006940,-37.807904,144.990201
std,0.971079,0.711362,897.449881,0.075850,0.099165
min,1.000000,1.000000,0.000000,-38.164920,144.542370
25%,2.000000,1.000000,152.000000,-37.855438,144.926198
50%,3.000000,1.000000,373.000000,-37.802250,144.995800
75%,4.000000,2.000000,628.000000,-37.758200,145.052700
max,8.000000,8.000000,37000.000000,-37.457090,145.526350


In [13]:
x.head(5) #top 5 rows of data in x

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
1,2,1.0,156.0,-37.8079,144.9934
2,3,2.0,134.0,-37.8093,144.9944
4,4,1.0,120.0,-37.8072,144.9941
6,3,2.0,245.0,-37.8024,144.9993
7,2,1.0,256.0,-37.8060,144.9954


In [14]:
from sklearn.tree import DecisionTreeRegressor #decision tree
# Define model
df_model = DecisionTreeRegressor(random_state=1) #random state number tells to set to same state after every run.

# Fitting the model
df_model.fit(x, y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=1, splitter='best')

In [15]:
#now the fun part
#lets predict the prices for first 10 houses
#The predictions are:
predictions = df_model.predict(x)
predictions

array([1035000., 1465000., 1600000., ...,  385000.,  560000., 2450000.])

In [16]:
#validating the model prediction by Mean Absolute Error
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y,predictions)



1115.7467183128902

In [17]:
#splitting data into train and validation
from sklearn.model_selection import train_test_split
train_x,val_x,train_y,val_y = train_test_split(x,y,random_state = 0)
df_model = DecisionTreeRegressor()
df_model.fit(train_x,train_y)
predictions = df_model.predict(val_x)
print(mean_absolute_error(val_y,predictions))


278463.62556488055


In [18]:
#checking for optimal leaf nodes
def get_mae(node,train_x,val_x,train_y,val_y):
  df_model = DecisionTreeRegressor(max_leaf_nodes= node, random_state= 0) #specify the model
  df_model.fit(train_x,train_y)#fitting the model
  predictions = df_model.predict(val_x)
  mae = mean_absolute_error(val_y,predictions)
  return mae
candidate_nodes = [5,50,500,5000]
for node in candidate_nodes:
  my_mae = get_mae(node,train_x,val_x,train_y,val_y)
  print("for %d nodes we have mean absolute error as: %d" %(node,my_mae)) 
score = {leaf_size: get_mae(leaf_size,train_x,val_x,train_y,val_y) for leaf_size in candidate_nodes }
  # min nodes required for optimal prediction is
min_node = min(score,key = score.get)
min_node


for 5 nodes we have mean absolute error as: 385696
for 50 nodes we have mean absolute error as: 279794
for 500 nodes we have mean absolute error as: 261718
for 5000 nodes we have mean absolute error as: 271996


500

In [19]:
df_model = DecisionTreeRegressor(max_leaf_nodes= min_node, random_state= 0) # since we know the optimal node value
df_model.fit(x,y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=500,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=0, splitter='best')

In [20]:
from sklearn.ensemble import RandomForestRegressor

In [21]:
df_model1 = RandomForestRegressor(random_state=0)
df_model1.fit(train_x,train_y)
predictionsRandom = df_model1.predict(val_x)
print('The mae for RandomForests is {}'.format(mean_absolute_error(val_y,predictionsRandom)))

The mae for RandomForests is 206868.39967967046


In [22]:
#clearly random forests gave better value than the decision tree